In [5]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-08-07 00:33:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  32.1MB/s    in 4.4s    

2020-08-07 00:33:33 (32.1 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [6]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-08-07 00:33:35--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 108.177.97.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  --.-KB/s    in 0.1s    

2020-08-07 00:33:35 (93.2 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [7]:
#Zip off data
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [8]:
#1)Data preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Directory of data
train_dir = os.path.join('/tmp/horse-or-human')
test_dir = os.path.join('/tmp/validation-horse-or-human')

#Normalize the pixels
train_datagen = ImageDataGenerator(rescale= 1/255)
test_datagen = ImageDataGenerator(rescale= 1/255)

#Images flow in from the training directory
train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size = (150,150), # all images resize to 150x150
                                                        batch_size = 128, # how big are the batches? 32, 64, 128 are usually good numbers
                                                        class_mode = 'binary') # binary classification

test_generator = test_datagen.flow_from_directory(test_dir,
                                                        target_size = (150,150), 
                                                        batch_size = 128, 
                                                        class_mode = 'binary') # binary classification

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Add convolutional layers as in the previous example, and flatten the final result to feed into the densely connected layers.

Finally we add the densely connected layers.

*Note that we are facing a two-class classification problem, i.e. a binary classification problem, we will end our network with a `sigmoid activation` so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [ ]:
#2)Build model
import tensorflow as tf
model = tf.keras.models.Sequential([#the first convolution
                                    tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #the second convolution
                                    tf.keras.layers.Conv2D(32, (3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #the third convolution
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #Flatten into DNN
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

#3)Compile
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy', #binary classification
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 18496)            

The "output shape" column shows the size of the feature map in each layer. The convolution layer reduced size of the feature maps by padding, and each pooling layer halves the dimensions.

Next, we'll configure the specifications for model training. We will train our model with the binary_crossentropy loss, because it's a binary classification problem and our final activation is a sigmoid. We will use the rmsprop optimizer with a learning rate of 0.001. During training, we will want to monitor classification accuracy.

NOTE: Using the RMSprop optimization algorithm is preferable to stochastic gradient descent (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as Adam and Adagrad, also automatically adapt the learning rate during training, and would work equally well here.)

*Optimizers like RMSProp, Adam and Adagrad all adaptively change the learning rate. It's important to adjust the learning rate as a model learns more, too big and the gradients could explode, too small and the gradients could vanish.

In [ ]:
#4)Training the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = test_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 21s 3s/step - loss: 4.0173 - acc: 0.5006 - val_loss: 0.6415 - val_acc: 0.5234
Epoch 2/15
8/8 [==============================] - 21s 3s/step - loss: 0.7870 - acc: 0.6741 - val_loss: 0.6446 - val_acc: 0.5586
Epoch 3/15
8/8 [==============================] - 22s 3s/step - loss: 0.4502 - acc: 0.8265 - val_loss: 1.6059 - val_acc: 0.5508
Epoch 4/15
8/8 [==============================] - 22s 3s/step - loss: 0.2486 - acc: 0.8799 - val_loss: 0.7724 - val_acc: 0.8164
Epoch 5/15
8/8 [==============================] - 21s 3s/step - loss: 0.1531 - acc: 0.9466 - val_loss: 0.3931 - val_acc: 0.8125
Epoch 6/15
8/8 [==============================] - 22s 3s/step - loss: 0.1975 - acc: 0.9099 - val_loss: 3.2851 - val_acc: 0.5352
Epoch 7/15
8/8 [==============================] - 22s 3s/step - loss: 0.3590 - acc: 0.8576 - val_loss: 0.9738 - val_acc: 0.8203
Epoch 8/15
8/8 [==============================] - 24s 3s/step - loss: 0.0453 - acc: 0.9855 - val_loss: 1

*fit_generator() should use instead of fit() when we are using image generators. 

In [ ]:
#5)Running Model
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for i in uploaded.keys():
  img = image.load_img('/content'+i, target_size=(150,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

Using TensorFlow backend.


# Exercise 4

In [10]:
import tensorflow as tf
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.99):
                print("\nReached 99% accuracy so cancelling training!")
                self.model.stop_training = True
                    

    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #the second convolution
                                    tf.keras.layers.Conv2D(32, (3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #the third convolution
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #Flatten into DNN
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')       
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001),metrics=['acc'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale = 1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (150,150),
        batch_size = 128,
        class_mode = 'binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=10,
      verbose=1)
    # model fitting
    return history.history['acc'][-1]
train_happy_sad_model()

Found 1027 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 18s 2s/step - loss: 1.3142 - acc: 0.5695
Epoch 2/15
8/8 [==============================] - 17s 2s/step - loss: 0.3730 - acc: 0.8409
Epoch 3/15
8/8 [==============================] - 20s 3s/step - loss: 0.2119 - acc: 0.9088
Epoch 4/15
8/8 [==============================] - 17s 2s/step - loss: 0.0660 - acc: 0.9844
Epoch 5/15
8/8 [==============================] - 17s 2s/step - loss: 0.2704 - acc: 0.8921
Epoch 6/15
8/8 [==============================] - 17s 2s/step - loss: 0.0543 - acc: 0.9811
Epoch 7/15
8/8 [==============================] - 17s 2s/step - loss: 0.0514 - acc: 0.9789
Epoch 8/15
8/8 [==============================] - 17s 2s/step - loss: 0.0416 - acc: 0.9867
Epoch 9/15
8/8 [==============================] - 17s 2s/step - loss: 0.2504 - acc: 0.9177
Epoch 10/15
8/8 [==============================] - 19s 2s/step 

1.0